### 0. Load environment variables

In [1]:
import os
import sys  
sys.path.append("..")  

from dotenv import load_dotenv
_ = load_dotenv('.env')

### 1. Get a random persona

In [2]:
from user_persona import UserPersona, get_random_user_persona
persona = get_random_user_persona().json()
persona

'{"gender": "female", "name": "Jean Forbes", "language": "English", "nationality": "Belgium", "age": 32, "hobbies": ["knitting", "pottery", "dancing", "sports", "gardening"], "talkative": true, "characteristics": ["NotWitty"], "education_level": "middle_school_level"}'

### 2. Get OpenAI functions
This prompt is supposed to work with OpenAI Functions 

In [3]:
from user_persona import UserProfile
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
openai_function_profile = convert_pydantic_to_openai_function(UserProfile)

### You can also get openai_function_profile in this way
# with open('openai_function_profile.json', 'r') as f:  
#     openai_function_profile = json.load(f)

### 3. Fetch the prompt and create a chain (using LCEL)


In [9]:
from typing import Literal
from langchain import hub
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

def get_OpenAI_chain(provider: Literal['ChatOpenAI', 'AzureChatOpenAI'], model_or_deployment_name: str):
    if provider == 'ChatOpenAI':
        chat_model = ChatOpenAI(model_name=model_or_deployment_name, temperature=0).bind(functions=[openai_function_profile], function_call={'name':'UserProfile'})
    else:
        chat_model = AzureChatOpenAI(deployment_name=model_or_deployment_name, temperature=0).bind(functions=[openai_function_profile], function_call={'name':'UserProfile'})
    return chat_model

prompt = hub.pull("jet-taekyo-lee/persona-description")
provider = 'AzureChatOpenAI'
model_or_deployment_name = os.getenv('model_name') if provider == 'ChatOpenAI' else os.getenv('deployment_name')
chat_model = get_OpenAI_chain(provider, model_or_deployment_name)

chain = prompt | chat_model

### 4. Get a response

In [10]:
response = chain.invoke({'persona':persona})
print(response.additional_kwargs['function_call']['arguments'])

{
  "original_persona": {
    "gender": "female",
    "name": "Jean Forbes",
    "language": "English",
    "nationality": "Belgium",
    "age": 32,
    "hobbies": ["knitting", "pottery", "dancing", "sports", "gardening"],
    "talkative": true,
    "characteristics": ["NotWitty"],
    "education_level": "middle_school_level"
  },
  "refined_persona": {
    "gender": "female",
    "name": "Jean Forbes",
    "language": "English",
    "nationality": "Belgium",
    "age": 32,
    "hobbies": ["knitting", "pottery", "dancing", "sports", "gardening"],
    "talkative": true,
    "characteristics": ["NotWitty"],
    "education_level": "middle_school_level"
  },
  "refined_persona_description": {
    "description_of_persona": "Jean Forbes is a 32-year-old woman from Belgium. She is fluent in English and is quite talkative. Despite having only a middle school level education, Jean has a wide range of hobbies that include knitting, pottery, dancing, sports, and gardening. One notable characteris